In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from metro_app.ml_logic.preprocess import general_preprocessing


## Goal of this notebook: 
See how I can filter for line, hour and entry vs exit 

To do so, I need to: 
1. create a first function that plot the dataframe and pass it into my raw df 

In [3]:
# 1) I preprocess my dataframe
data = pd.read_csv('passenger_flow.csv')
new_data = general_preprocessing(data)


/var/folders/r3/6gpwk0t13dngmyc7b_0pcjb40000gn/T/ipykernel_1803/2584142203.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('flows.csv')
/Users/yannickdeza/code/wagon_project/metro_seoul/metro_app/ml_logic/preprocess.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = pd.to_datetime(df['date'])
/Users/yannickdeza/code/wagon_project/metro_seoul/metro_app/ml_logic/preprocess.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

In [4]:
# 2) I filter for a specific date
new_data = data[data['date'] == '2023-06-30']


In [5]:
# 3) I drop columns I don't care about

new_data.drop(columns=['Unnamed: 0', 'station_number', 'date', '24', '01', '02', '03', '04'], inplace=True)
new_data.drop_duplicates(inplace=True)
new_data


/var/folders/r3/6gpwk0t13dngmyc7b_0pcjb40000gn/T/ipykernel_1803/1662240452.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data.drop(columns=['Unnamed: 0', 'station_number', 'date', '24', '01', '02', '03', '04'], inplace=True)
/var/folders/r3/6gpwk0t13dngmyc7b_0pcjb40000gn/T/ipykernel_1803/1662240452.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data.drop_duplicates(inplace=True)


,line,station_name,entry/exit,05,06,07,08,09,10,11,...,14,15,16,17,18,19,20,21,22,23
0,1,Sinseoldong,entry,342,362,946,1238,758,719,745,...,827,951,1073,1664,2254,906,631,484,345,195
1,1,Sinseoldong,exit,93,307,1021,2474,1412,881,652,...,690,683,784,894,1107,846,479,503,462,414
2,1,Seoul Station,exit,293,1693,4523,9700,6086,2913,2418,...,2601,2819,2982,3798,4346,3249,1854,1932,1129,660
3,1,Seoul Station,entry,280,498,1672,2541,2140,1972,2553,...,2630,4113,5016,7099,9391,4576,3286,3501,2782,1153
4,1,Jongno 5(o)ga,entry,73,118,230,359,483,866,1372,...,2236,2309,2522,3481,4070,1952,1297,1305,972,333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541,8,Cheonho(Pungnaptoseong),exit,51,425,572,1193,934,682,731,...,869,1036,1188,1785,2971,1961,1045,918,848,651
542,8,Bokjeong,exit,84,337,459,635,325,240,213,...,295,344,493,820,1164,840,473,394,455,257
543,8,Bokjeong,entry,127,444,1176,1537,714,411,328,...,330,402,645,837,1005,507,308,264,227,193
544,8,Amsa,entry,608,1222,3661,4349,1973,1048,842,...,696,759,785,1021,1013,622,395,340,221,110


## Now I work on the two other dataframes: 
1. Coordinates data 
2. Translation data 

### Translation data pre-processing

In [6]:
# 1) I upload and clean the translation dataframe
map = pd.read_excel('translation_data.xlsx')
map.drop(map.columns[0:2], axis=1, inplace=True)
map.rename(columns={map.columns[1]: 'station_name', map.columns[0]: 'korean_name'}, inplace=True)
map


,korean_name,station_name
0,서울역,Seoul Station
1,시청,City Hall
2,종각,Jonggak
3,종로3가,Jongno 3(sam)ga
4,종로5가,Jongno 5(o)ga
...,...,...
293,송파나루,Songpanaru
294,한성백제,Hanseong Baekje
295,올림픽공원(한국체대),Olympic Park(Korea National Sport University)
296,둔촌오륜,Dunchon Oryun


In [7]:
# 2) I remove duplicates
map.drop_duplicates(inplace=True)
map


,korean_name,station_name
0,서울역,Seoul Station
1,시청,City Hall
2,종각,Jonggak
3,종로3가,Jongno 3(sam)ga
4,종로5가,Jongno 5(o)ga
...,...,...
291,석촌고분,Seokchon Gobun
293,송파나루,Songpanaru
294,한성백제,Hanseong Baekje
296,둔촌오륜,Dunchon Oryun


### Location data pre-processing

In [8]:
# I check location data
loc = pd.read_csv('location_data.csv')
loc.drop(columns=['Unnamed: 0'], inplace=True)
loc.rename(columns={'name': 'korean_name'}, inplace=True)
loc


,line,korean_name,lat,lng,no
0,01호선,소요산,37.948100,127.061034,100
1,01호선,동두천,37.927878,127.054790,101
2,01호선,보산,37.913702,127.057277,102
3,01호선,동두천중앙,37.901885,127.056482,103
4,01호선,지행,37.892334,127.055716,104
...,...,...,...,...,...
704,경강선,신둔도예촌,37.317185,127.404760,K416
705,경강선,이천,37.265579,127.442260,K417
706,경강선,부발,37.260192,127.490277,K418
707,경강선,세종대왕릉,37.295309,127.570938,K419


In [9]:
# I clean the "line" column and I only keep lines from 1 to 8
loc['line'] = loc.line.str.extract('(\d+)')
loc['line'].replace('2', np.nan, inplace=True)
loc.dropna(inplace=True)
loc['line'] = loc.line.str.lstrip('0')
loc['line'].replace('9', np.nan, inplace=True)
loc.dropna(inplace=True)
loc


,line,korean_name,lat,lng,no
0,1,소요산,37.948100,127.061034,100
1,1,동두천,37.927878,127.054790,101
2,1,보산,37.913702,127.057277,102
3,1,동두천중앙,37.901885,127.056482,103
4,1,지행,37.892334,127.055716,104
...,...,...,...,...,...
399,8,남한산성입구,37.451535,127.159816,822
400,8,단대오거리,37.445210,127.156866,823
401,8,신흥,37.440918,127.147564,824
402,8,수진,37.437428,127.140722,825


### Merging translation and location data

In [80]:
merged = loc.merge(map, on='korean_name')
merged


,line,korean_name,lat,lng,no,station_name
0,1,도봉산,37.689313,127.046222,113,Dobongsan
1,7,도봉산,37.689313,127.046222,710,Dobongsan
2,1,창동,37.653166,127.047731,116,Changdong
3,4,창동,37.653166,127.047731,412,Changdong
4,1,석계,37.614805,127.065851,120,Seokgye
...,...,...,...,...,...,...
219,8,산성,37.457122,127.149908,821,Sanseong
220,8,단대오거리,37.445210,127.156866,823,Dandaeogeori
221,8,신흥,37.440918,127.147564,824,Sinheung
222,8,수진,37.437428,127.140722,825,Sujin


In [81]:
merged.drop_duplicates(inplace=True)
merged


,line,korean_name,lat,lng,no,station_name
0,1,도봉산,37.689313,127.046222,113,Dobongsan
1,7,도봉산,37.689313,127.046222,710,Dobongsan
2,1,창동,37.653166,127.047731,116,Changdong
3,4,창동,37.653166,127.047731,412,Changdong
4,1,석계,37.614805,127.065851,120,Seokgye
...,...,...,...,...,...,...
219,8,산성,37.457122,127.149908,821,Sanseong
220,8,단대오거리,37.445210,127.156866,823,Dandaeogeori
221,8,신흥,37.440918,127.147564,824,Sinheung
222,8,수진,37.437428,127.140722,825,Sujin


In [82]:
print(f'In the entry/exit dataframe, I have {len(new_data.station_name.unique())} unique stations')
print(f'In the location dataframe, I have {len(merged.station_name.unique())} unique stations')
missing = [el for el in
      new_data.station_name.unique().tolist()
      if el not in merged.station_name.unique().tolist()]
n_missing = len(missing)
print(f'There are {n_missing} stations in the entry/exit dataframe for which we do not have location data')


In the entry/exit dataframe, I have 239 unique stations
In the location dataframe, I have 190 unique stations
There are 59 stations in the entry/exit dataframe for which we do not have location data


In [83]:
merged.rename(columns={'no': 'station_number'}, inplace=True)
merged


,line,korean_name,lat,lng,station_number,station_name
0,1,도봉산,37.689313,127.046222,113,Dobongsan
1,7,도봉산,37.689313,127.046222,710,Dobongsan
2,1,창동,37.653166,127.047731,116,Changdong
3,4,창동,37.653166,127.047731,412,Changdong
4,1,석계,37.614805,127.065851,120,Seokgye
...,...,...,...,...,...,...
219,8,산성,37.457122,127.149908,821,Sanseong
220,8,단대오거리,37.445210,127.156866,823,Dandaeogeori
221,8,신흥,37.440918,127.147564,824,Sinheung
222,8,수진,37.437428,127.140722,825,Sujin


In [93]:
# I will remove the line info, so that I don t have duplicates

# merged.drop(columns=['line', 'station_number'], inplace=True)
merged.drop_duplicates(inplace=True)
merged.sort_values(by=['line', 'station_number'], inplace=True)
merged


,line,korean_name,lat,lng,station_number,station_name
0,1,도봉산,37.689313,127.046222,113,Dobongsan
2,1,창동,37.653166,127.047731,116,Changdong
4,1,석계,37.614805,127.065851,120,Seokgye
6,1,제기동,37.578103,127.034893,125,Jegidong
7,1,신설동,37.575297,127.025087,126,Sinseoldong
...,...,...,...,...,...,...
219,8,산성,37.457122,127.149908,821,Sanseong
220,8,단대오거리,37.445210,127.156866,823,Dandaeogeori
221,8,신흥,37.440918,127.147564,824,Sinheung
222,8,수진,37.437428,127.140722,825,Sujin


In [110]:
# Here I create a Dataframe with lines

filter_df = merged.drop(columns=['korean_name'])
# line_1 = filter_df[filter_df['line'] == '1']
# line_1['next_lat'] = line_1['lat'].shift(-1, axis=0)
# line_1['next_lng'] = line_1['lng'].shift(-1, axis=0)
# line_1

lines = {}

for i in range(1, 9, 1):
    line = filter_df[filter_df['line'] == f'{i}']
    line['next_lat'] = line['lat'].shift(-1, axis=0)
    line['next_lng'] = line['lng'].shift(-1, axis=0)

    lines[f'line_{i}'] = line

lines


/var/folders/r3/6gpwk0t13dngmyc7b_0pcjb40000gn/T/ipykernel_1803/3374144571.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/r3/6gpwk0t13dngmyc7b_0pcjb40000gn/T/ipykernel_1803/3374144571.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/r3/6gpwk0t13dngmyc7b_0pcjb40000gn/T/ipykernel_1803/3374144571.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

{'line_1':    line        lat         lng station_number           station_name  \
 0     1  37.689313  127.046222            113              Dobongsan   
 2     1  37.653166  127.047731            116              Changdong   
 4     1  37.614805  127.065851            120                Seokgye   
 6     1  37.578103  127.034893            125               Jegidong   
 7     1  37.575297  127.025087            126            Sinseoldong   
 9     1  37.572627  127.016429            127                Dongmyo   
 11    1  37.571420  127.009745            128             Dongdaemun   
 13    1  37.570926  127.001849            129          Jongno 5(o)ga   
 14    1  37.571607  126.991806            130        Jongno 3(sam)ga   
 17    1  37.570161  126.982923            131                Jonggak   
 18    1  37.564718  126.977108            132              City Hall   
 20    1  37.554648  126.972559            133          Seoul Station   
 22    1  37.517122  126.917169          

### I am ready to merge the two dataframes

In [89]:
complete = new_data.merge(merged, on='station_name')
complete.drop(columns=['korean_name'], inplace=True)
complete


,line_x,station_name,entry/exit,05,06,07,08,09,10,11,...,18,19,20,21,22,23,line_y,lat,lng,station_number
0,1,Sinseoldong,entry,342,362,946,1238,758,719,745,...,2254,906,631,484,345,195,1,37.575297,127.025087,126
1,1,Sinseoldong,entry,342,362,946,1238,758,719,745,...,2254,906,631,484,345,195,2,37.575297,127.025087,211-4
2,1,Sinseoldong,exit,93,307,1021,2474,1412,881,652,...,1107,846,479,503,462,414,1,37.575297,127.025087,126
3,1,Sinseoldong,exit,93,307,1021,2474,1412,881,652,...,1107,846,479,503,462,414,2,37.575297,127.025087,211-4
4,2,Sinseoldong,exit,13,70,218,503,314,275,220,...,288,171,139,121,128,89,1,37.575297,127.025087,126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527,8,Dandaeogeori,exit,54,218,484,818,344,329,357,...,1859,1530,744,700,794,497,8,37.445210,127.156866,823
528,8,Bokjeong,exit,84,337,459,635,325,240,213,...,1164,840,473,394,455,257,8,37.470047,127.126662,820
529,8,Bokjeong,entry,127,444,1176,1537,714,411,328,...,1005,507,308,264,227,193,8,37.470047,127.126662,820
530,8,Amsa,entry,608,1222,3661,4349,1973,1048,842,...,1013,622,395,340,221,110,8,37.550210,127.127562,810


In [90]:
columns_to_unpivot = complete.columns[3:22].tolist()
columns_to_unpivot


['05',
 '06',
 '07',
 '08',
 '09',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23']

In [91]:
final_df = pd.melt(complete, id_vars=['station_name', 'line', 'entry/exit', 'lat', 'lng'],
        value_vars=columns_to_unpivot)
final_df.rename(columns={'value': 'n_passengers', 'variable': 'hour'}, inplace=True)
final_df


KeyError: "The following 'id_vars' are not present in the DataFrame: ['line']"

## Defining the function 

In the function we want to have a visualization of the traffic given: 
1. The line 
2. The metro stop (more or less crowded)

In [20]:

import plotly.express as px
fig = px.density_mapbox(final_df, lat="lat", lon="lng", z='n_passengers', radius=13,
                        center=dict(lat=37.5519, lon=126.9918), zoom=10,animation_frame="hour",
                        color_continuous_scale = px.colors.sequential.Plasma,
                        hover_name = 'station_name',
                        range_color = [50, 1000],
                        mapbox_style="carto-positron")
fig.show()


In [21]:
final_df[(final_df['line'] == 2) & (final_df['entry/exit'] == 'entry')]


,station_name,line,entry/exit,lat,lng,hour,n_passengers
3,Sinseoldong,2,entry,37.575297,127.025087,05,58
30,City Hall,2,entry,37.564718,126.977108,05,24
33,Yongdap,2,entry,37.561904,127.050899,05,91
35,Yeongdeungpo-gu Office,2,entry,37.524970,126.895951,05,205
39,Yeoksam,2,entry,37.500622,127.036456,05,140
...,...,...,...,...,...,...,...
7487,Dorimcheon,2,entry,37.514287,126.882768,23,20
7489,Dangsan,2,entry,37.534380,126.902281,23,470
7491,Bongcheon,2,entry,37.482362,126.941892,23,217
7492,Bangbae,2,entry,37.481426,126.997596,23,332


In [22]:

final_df.loc[final_df['station_name'] == 'Sinseoldong', 'lat'][0]


37.575297

In [192]:
# Added a point marking my current position in last add_trace, need to adjust it

def pick_station(station, line):

    df = final_df[(final_df['line'] == line)]


    fig = px.density_mapbox(df, lat="lat", lon="lng", z='n_passengers', radius=13,
                        center=dict(lat=37.5519, lon=126.9918), zoom=10,animation_frame="hour",
                        color_continuous_scale = px.colors.sequential.Plasma_r,
                        hover_name = 'station_name',
                        range_color = [50, 1500],
                        mapbox_style="carto-positron")

    # I draw the line of the metro

    for _,row in lines[f'line_{line}'].iterrows():
        fig.add_trace(go.Scattermapbox(mode='lines',
                                lon=[row['lng'], row['next_lng']],
                                lat=[row['lat'], row['next_lat']],
                                line_color='lightgrey',
                                line=dict(width=2),
                                name=None,
                                showlegend=False
                                ))

    lat = final_df.loc[final_df['station_name'] == station, 'lat'][0]
    lon = final_df.loc[final_df['station_name'] == station, 'lng'][0]

    # I draw the point where I am right now


    fig.add_trace(go.Scattermapbox(
        lat=[lat],
                lon=[lon],
                mode='markers',
                marker=dict(size=8, color='salmon'),
                showlegend=False))


    # I draw the points of the metro

    fig.add_trace(go.Scattermapbox(
            lat=df['lat'],
            lon=df['lng'],
            mode='markers',
            marker=dict(size=4, color='lightsalmon'),
            showlegend=False))




    fig.show()


In [193]:
pick_station('Sinseoldong', 6)


In [143]:
import plotly.express as px

print(px.colors.sequential.ice_r)


['rgb(234, 252, 253)', 'rgb(192, 229, 232)', 'rgb(149, 207, 216)', 'rgb(114, 184, 205)', 'rgb(89, 159, 196)', 'rgb(72, 134, 187)', 'rgb(62, 109, 178)', 'rgb(62, 83, 160)', 'rgb(58, 60, 125)', 'rgb(44, 42, 87)', 'rgb(25, 25, 51)', 'rgb(3, 5, 18)']


In [116]:
import plotly.graph_objects as go

fig = go.Figure()


for _,row in lines['line_7'].iterrows():
    fig.add_trace(go.Scattermapbox(mode='lines',
                                   lon=[row['lng'], row['next_lng']],
                                   lat=[row['lat'], row['next_lat']],
                                   line_color='green',
                                   name=row['line']
                                  ))
fig.update_layout(
    height=600,
    mapbox=dict(
        style='open-street-map',
        zoom=10,
        center=dict(lon=df['lng'].mean(), lat=df['lat'].mean())
    )
)
fig.show()


In [ ]:
import geopy.distance

coords_1 = (52.2296756, 21.0122287)
coords_2 = (52.406374, 16.9251681)

print geopy.distance.geodesic(coords_1, coords_2).km


In [67]:
import plotly.graph_objects as go

# Sample data with latitude and longitude
data = dict(
    lat=[40.7128, 34.0522, 41.8781],
    lon=[-74.0060, -118.2437, -87.6298],
    names=['New York', 'Los Angeles', 'Chicago']
)

# Create a scatter mapbox plot with points
fig = go.Figure()

# Add scatter plot for points
fig.add_trace(go.Scattermapbox(
    lat=data['lat'],
    lon=data['lon'],
    mode='markers',
    marker=dict(size=10, color='blue'),
    text=data['names']
))

# Add scatter plot for lines
fig.add_trace(go.Scattermapbox(
    lat=data['lat'] + [data['lat'][0]],
    lon=data['lon'] + [data['lon'][0]],
    mode='lines',
    line=dict(width=2, color='red'),
    text=data['names'] + [data['names'][0]],
    hoverinfo='text'
))

# Update layout to use Mapbox style
fig.update_layout(
    mapbox=dict(
        style="open-street-map",
        center=dict(lat=data['lat'][0], lon=data['lon'][0]),
        zoom=3
    )
)

# Show the figure
fig.show()


In [ ]:
for line_type in final_df['line'].unique():
    line_data = final_df[final_df['line'] == line_type]
    fig.add_trace(go.Scattermapbox(
        lat=line_data['lat'],
        lon=line_data['lng'],
        mode='markers',
        line=dict(width=2, color=line_data.color.max()),
        marker=dict(size=10, color=line_data.color.max()),
        text=[f'Point {i + 1}, Line: {line}' for i, line in enumerate(line_data['line'])],
        name=line_type  # Utilisez le nom de la ligne comme étiquette pour la légende
    ))
fig.show()


In [247]:
line_data = final_df[final_df['line'] == 1]
fig = go.Figure(go.Choroplethmapbox())
fig.update_layout(mapbox_style="carto-positron",
                  mapbox_zoom=10.5, mapbox_center = {"lon":126.9779692, "lat":37.536535})
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.add_trace(go.Scattermapbox(
        lat=line_data['lat'],
        lon=line_data['lng'],
        mode='markers',
        line=dict(width=2, color='yellow'),
        marker=dict(size=10, color='yellow'),
        text=[f'Point {i + 1}, Line: {line}' for i, line in enumerate(line_data['line'])],
        name='line 1'  # Utilisez le nom de la ligne comme étiquette pour la légende
    ))


fig.show()


In [260]:
from plotly.validators.scatter.marker import SymbolValidator

raw_symbols = SymbolValidator().values

raw_symbols


[0,
 '0',
 'circle',
 100,
 '100',
 'circle-open',
 200,
 '200',
 'circle-dot',
 300,
 '300',
 'circle-open-dot',
 1,
 '1',
 'square',
 101,
 '101',
 'square-open',
 201,
 '201',
 'square-dot',
 301,
 '301',
 'square-open-dot',
 2,
 '2',
 'diamond',
 102,
 '102',
 'diamond-open',
 202,
 '202',
 'diamond-dot',
 302,
 '302',
 'diamond-open-dot',
 3,
 '3',
 'cross',
 103,
 '103',
 'cross-open',
 203,
 '203',
 'cross-dot',
 303,
 '303',
 'cross-open-dot',
 4,
 '4',
 'x',
 104,
 '104',
 'x-open',
 204,
 '204',
 'x-dot',
 304,
 '304',
 'x-open-dot',
 5,
 '5',
 'triangle-up',
 105,
 '105',
 'triangle-up-open',
 205,
 '205',
 'triangle-up-dot',
 305,
 '305',
 'triangle-up-open-dot',
 6,
 '6',
 'triangle-down',
 106,
 '106',
 'triangle-down-open',
 206,
 '206',
 'triangle-down-dot',
 306,
 '306',
 'triangle-down-open-dot',
 7,
 '7',
 'triangle-left',
 107,
 '107',
 'triangle-left-open',
 207,
 '207',
 'triangle-left-dot',
 307,
 '307',
 'triangle-left-open-dot',
 8,
 '8',
 'triangle-right',
 10